#Indexação do arquivo de telegramas do Wikileaks

Nesta prática vamos indexar todo oarquivo de telegramas do Wikileaks com o elasticsearch e testar algumas de suas características como sistema de recuperação de informação.

Para realizar esta prática, é necessário fazer o Download do seguinte arquivo:
https://archive.org/details/wikileaks-cables-csv

Também é necessário instalar a biblioteca cablemap: https://github.com/heuer/cablemap


In [1]:
from cablemap.core import cables_from_source
from elasticsearch import Elasticsearch as ES
from elasticsearch import helpers
import datetime
from collections import defaultdict

Primeiro temos que localizar o arquivo com os telegramas.

In [2]:
fname = "/home/fccoelho/Downloads/wikileaks_cables/cables.csv"

In [3]:
i = 0
for cable in cables_from_source(fname):
    print(cable.subject, cable.created)
    if i>5:
        break
    i += 1

(u'EXTENDED NATIONAL JURISDICTIONS OVER HIGH SEAS', u'1966-12-28 18:48')
(u'ACCELERATION OF F-4ES FOR IRAN', u'1972-02-25 09:30')
(u'TRIALS/EXECUTIONS OF ANTI-GOVERNMENT ELEMENTS: STUDENTS DEMONSTRATE AND SHAH LASHES OUT AT FOREIGN CRITICS', u'1972-03-09 05:40')
(u'CONTINUING TERRORIST ACTIVITIES IN IRAN', u'1972-08-10 04:00')
(u'CONTINUING TERRORIST VIOLENCE', u'1972-08-22 09:27')
(u'AUDIENCE WITH SHAH APRIL 5', u'1973-04-02 08:34')
(u'ASSASSINATION/KIDNAP PLOT AGAINST SHAH REVEALED', u'1973-10-02 14:00')


In [4]:
dir(cable)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__implemented__',
 '__init__',
 '__module__',
 '__new__',
 '__providedBy__',
 '__provides__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 'cabledrum_uri',
 'canonical_id',
 'classification',
 'classification_categories',
 'classified_by',
 'comment',
 'content',
 'created',
 'header',
 'info_recipients',
 'is_partial',
 'nondisclosure_deadline',
 'origin',
 'plusd_canonical_id',
 'plusd_uri',
 'recipients',
 'reference_id',
 'references',
 'released',
 'signed_by',
 'subject',
 'summary',
 'tags',
 'transmission_id',
 'wl_uris']

Cada telegrama tem vários atributos, por isso precisamos construir um documento JSON com todas estes atributos

Vamos ignorar dois atributos por causa de bugs na biblioteca cablemap

In [5]:
atributos = [i for i in dir(cable) if not i.startswith('_')]
print atributos
atributos.pop(3) #remove "classification_categories"
atributos.pop(4) #remove "comment"
print atributos

['cabledrum_uri', 'canonical_id', 'classification', 'classification_categories', 'classified_by', 'comment', 'content', 'created', 'header', 'info_recipients', 'is_partial', 'nondisclosure_deadline', 'origin', 'plusd_canonical_id', 'plusd_uri', 'recipients', 'reference_id', 'references', 'released', 'signed_by', 'subject', 'summary', 'tags', 'transmission_id', 'wl_uris']
['cabledrum_uri', 'canonical_id', 'classification', 'classified_by', 'content', 'created', 'header', 'info_recipients', 'is_partial', 'nondisclosure_deadline', 'origin', 'plusd_canonical_id', 'plusd_uri', 'recipients', 'reference_id', 'references', 'released', 'signed_by', 'subject', 'summary', 'tags', 'transmission_id', 'wl_uris']


In [6]:
getattr(cable,'cabledrum_uri')

u'http://www.cabledrum.net/cables/73TEHRAN7005'

Agora temos que configurar o Elasticsearch para receber a nossa coleção:

In [7]:
es = ES()
es.indices.create(index='wikileaks', ignore=400)

{u'error': u'IndexAlreadyExistsException[[wikileaks] already exists]',
 u'status': 400}

In [8]:
def build_doc(cable):
    doc = {}
    for a in atributos:
        try:
            if a == 'created':
                doc[a] = datetime.datetime.strptime(getattr(cable, a).strip(), "%Y-%m-%d %H:%M")
            else:
                doc[a] = getattr(cable, a)
        except AttributeError as e:
            print e
            doc[a] = ""
    return doc

def cable_doc_gen():
    """
    Função geradora que itera sobre cables.csv
    retornando um telegrama por vez, incluindo-o em um dicionário compatível com o elasticsearch.
    """
    for j,cable in enumerate(cables_from_source(fname)):
        doc = build_doc(cable)
        
        action = {
            "_index": "wikileaks",
            "_type": "telegramas",
            "_id": j,
            "doc": doc
            }
        if j%1000 == 0:
            print "Indexando telegrama número {}".format(j)
        yield action
        
helpers.bulk(es, cable_doc_gen(), chunk_size=1000)

FM SECSTATE WASHDC
INFO AMEMBASSY ATHENS
"
FM SECSTATE WASHDC
INFO USDEL SECRETARY IMMEDIATE
"
FM SECSTATE WASHDC
INFO USMISSION GENEVA IMMEDIATE
AMEMBASSY HELSINKI IMMEDIATE
"
FM SECSTATE WASHDC
INFO ALL AMERICAN REPUBLIC DIPLOMATIC POSTS IMMEDIATE
"
FM SECSTATE WASHDC
INFO AMEMBASSY KUWAIT IMMEDIATE
"
FM SECSTATE WASHDC
INFO USDEL SECRETARY IMMEDIATE
"
FM SECSTATE WASHDC
INFO USDEL SECRETARY IMMEDIATE
"
FM SECSTATE WASHDC
INFO USDEL SECRETARY IMMEDIATE 
"


Indexando telegrama número 0
'Reference' object has no attribute 'name'
Indexando telegrama número 1000

FM SECSTATE WASHDC
INFO ALL DIPLOMATIC AND CONSULAR POSTS
SPECIAL EMBASSY PROGRAM
USOFFICE PRISTINA 
AMEMBASSY DUSHANBE 
AMEMBASSY BELGRADE 
AMEMBASSY FREETOWN "



'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'

FM SECSTATE WASHDC
INFO ALL DIPLOMATIC AND CONSULAR POSTS
SPECIAL EMBASSY PROGRAM
AMEMBASSY FREETOWN 
USOFFICE PRISTINA 
AMEMBASSY DUSHANBE 
AMEMBASSY BELGRADE "


...

PP RUEHLMC
DE RUEHDR #0111 0350944
ZNR UUUUU ZZH ZFR ZFR ZFR
P 040944Z FEB 10
FM AMEMBASSY DAR ES SALAAM
RUEHLGB/AMEMBASSY KIGALI 1640
RUEHNR/AMEMBASSY NAIROBI 1608
RUEHNY/AMEMBASSY OSLO 0028
RUEHDS/AMEMBASSY ADDIS ABABA 3369
RHMFISS/CDR USAFRICOM STUTTGART GE
RHMFIUU/CJTF HOA//J3
RUEAIIA/CIA WASHINGTON DC
RUEHLMC/MCC WASHINGTON DC
RUEHPH/CDC ATLANTA GA"



'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama número 248000
'Reference' object has no attribute 'name'
Indexando telegrama número 249000
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama número 250000
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'

RR RUEHBZ RUEHDU RUEHGI RUEHJO RUEHMA RUEHMR RUEHPA RUEHRN RUEHTRO
DE RUEHAN #0113/01 0570417
ZNR UUUUU ZZH ANTANANARIVO
R 260417Z FEB 10
FM SECSTATE WASHDC
INFO RUEHZO/AFRICAN UNION COLLECTIVE
RUCNDT/USMISSION USUN NEW YORK
RUEHBS/USEU BRUSSELS
RHMFISS/HQ USAFRICOM STUTTGART GE
RUEATRS/DEPT OF TREASURY WASHDC
RUEAIIA/CIA WASHDC
RHMFIUU/CJTF HOA
RHEHNSC/NSC WASHDC
AMEMBASSY"



Indexando telegrama número 251000
'Reference' object has no attribute 'name'


(251287, [])

##Realizando consultas no índice

In [11]:
es.search("wikileaks", q='lula')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'84955',
    u'_index': u'wikileaks',
    u'_score': 0.9305093,
    u'_source': {u'doc': {u'cabledrum_uri': u'http://www.cabledrum.net/cables/06BRASILIA2352',
      u'canonical_id': u'06BRASILIA2352',
      u'classification': u'CONFIDENTIAL',
      u'classified_by': [u'Dennis Hearne'],
      u'content': u'C O N F I D E N T I A L SECTION 01 OF 02 BRASILIA 002352 \n \nSIPDIS \n \nSIPDIS \n \nE.O. 12958: DECL: 11/07/2016 \nTAGS: PREL, BR \nSUBJECT: BRAZIL: AMBASSADOR SOBEL PRESENTS CREDENTIALS TO \nPRESIDENT LULA, 7 NOVEMBER 2006 \n \n \nClassified By: ACTING DCM DENNIS HEARNE. REASONS: 1.4 (B)(D). \n \n1. (C) Summary. Ambassador Sobel, who arrived in Brazil in \nAugust, presented his credentials to President Lula da Silva \non 7 November, in a ceremony long delayed by Brazil\'s \nelection campaign season.  The 30-minute meeting with Lula \nwas positive and substantive, with warm atmospherics.  Prio

In [18]:
es.search?

In [14]:
cable.tags

[u'PINS', u'IR']

In [15]:
import requests

In [16]:
consulta = {
    "constant_score" : {
        "filter" : {
            "exists" : { "field" : "CLASSIFIED" }
        }
    }
}

In [20]:
resp = requests.get('http://localhost:9200/wikileaks/', params=consulta)

In [21]:
resp.text

u'{"wikileaks":{"aliases":{},"mappings":{"telegramas":{"properties":{"doc":{"properties":{"cabledrum_uri":{"type":"string"},"canonical_id":{"type":"string"},"classification":{"type":"string"},"classified_by":{"type":"string"},"content":{"type":"string"},"created":{"type":"date","format":"dateOptionalTime"},"header":{"type":"string"},"info_recipients":{"type":"string"},"is_partial":{"type":"boolean"},"nondisclosure_deadline":{"type":"string"},"origin":{"type":"string"},"plusd_canonical_id":{"type":"string"},"plusd_uri":{"type":"string"},"recipients":{"type":"string"},"reference_id":{"type":"string"},"references":{"type":"string"},"signed_by":{"type":"string"},"subject":{"type":"string"},"summary":{"type":"string"},"tags":{"type":"string"},"transmission_id":{"type":"string"},"wl_uris":{"type":"string"}}}}}},"settings":{"index":{"creation_date":"1427829970504","uuid":"RfcJyOB6QrG9rn_L3SfT-A","number_of_replicas":"1","number_of_shards":"5","version":{"created":"1050099"}}},"warmers":{}}}'

#classificando os telegramas
primeiro vamos escolher um campo para a nossa classificação

In [6]:
count = 0
tipos = defaultdict(lambda:0)
for n, cable in enumerate(cables_from_source(fname)):
    if cable.classification != "":
        count += 1
        tipos[cable.classification] += 1
        
print count 
print tipos

251287
defaultdict(<function <lambda> at 0x7fa58fd4f140>, {u'CONFIDENTIAL': 97070, u'CONFIDENTIAL//NOFORN': 4678, u'UNCLASSIFIED': 75792, u'SECRET': 11322, u'SECRET//NOFORN': 4330, u'UNCLASSIFIED//FOR OFFICIAL USE ONLY': 58095})


Para treinar o classificador, vamos selecionar um subconjunto dos telegramas que caiba na memória.

In [7]:
dados = []
for n, cable in enumerate(cables_from_source(fname)):
    dados.append((cable.content, cable.classification))
    if n>5000:
        break
print len(dados)

5002


In [8]:
from pprint import pprint
import numpy as np
from time import time
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cross_validation import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from nltk.corpus import stopwords

In [9]:
dados_train, dados_test  = dados[:3000], dados[3000:]
print len(dados_train), len(dados_test)

3000 2002


In [10]:
t0 = time()

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')

X = vectorizer.fit_transform((d[0] for d in dados_train))
Y = np.array([d[1] for d in dados_train])
print("Completo em %fs" % (time() - t0))
print("N amostras: %d, N features: %d" % X.shape)
print Y[:10]

Completo em 2.700161s
N amostras: 3000, N features: 69082
[u'UNCLASSIFIED' u'UNCLASSIFIED' u'UNCLASSIFIED' u'UNCLASSIFIED'
 u'UNCLASSIFIED' u'SECRET' u'UNCLASSIFIED' u'SECRET' u'CONFIDENTIAL'
 u'CONFIDENTIAL']


Agora vamos extrair as features do conjunto de teste

In [11]:
t0 = time()

X_test = vectorizer.transform((d[0] for d in dados_test))
Y_test = [d[1] for d in dados_test]
print("Completo em %fs" % (time() - t0))
print("N amostras: %d, N features: %d" % X_test.shape)
print X_test.shape,len(Y_test),Y_test[:10]

Completo em 1.434324s
N amostras: 2002, N features: 69082
(2002, 69082) 2002 [u'UNCLASSIFIED', u'CONFIDENTIAL', u'UNCLASSIFIED', u'UNCLASSIFIED', u'UNCLASSIFIED//FOR OFFICIAL USE ONLY', u'CONFIDENTIAL', u'UNCLASSIFIED', u'UNCLASSIFIED//FOR OFFICIAL USE ONLY', u'CONFIDENTIAL', u'CONFIDENTIAL']


Agora podemos treinar e testar os classificadores

In [12]:
cls = KNeighborsClassifier(n_neighbors=10)
cls.fit(X,Y)
pred = cls.predict(X_test)
print len(pred), len(Y_test)
score = metrics.accuracy_score(Y_test, pred)
score

2002 2002


0.6048951048951049

In [13]:
print metrics.classification_report(Y_test, pred)

             precision    recall  f1-score   support

CONFIDENTIAL       0.58      0.79      0.67       790
CONFIDENTIAL//NOFORN       0.00      0.00      0.00        10
     SECRET       0.08      0.02      0.04        85
SECRET//NOFORN       0.00      0.00      0.00        31
UNCLASSIFIED       0.68      0.66      0.67       802
UNCLASSIFIED//FOR OFFICIAL USE ONLY       0.47      0.19      0.27       284

avg / total       0.57      0.60      0.57      2002



/usr/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1905: UserWarning: The sum of true positives and false positives are equal to zero for some labels. Precision is ill defined for those labels [u'CONFIDENTIAL//NOFORN' u'SECRET//NOFORN']. The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [u'CONFIDENTIAL//NOFORN' u'SECRET//NOFORN']. 
  average=None)


In [14]:
cls = RandomForestClassifier(n_estimators=1000)
cls.fit(X.toarray(),Y)
pred = cls.predict(X_test.toarray())
print len(pred), len(Y_test)
score = metrics.accuracy_score(Y_test, pred)
score

2002 2002


0.74675324675324672

In [15]:
print metrics.classification_report(Y_test, pred)

             precision    recall  f1-score   support

CONFIDENTIAL       0.71      1.00      0.83       790
CONFIDENTIAL//NOFORN       0.00      0.00      0.00        10
     SECRET       0.00      0.00      0.00        85
SECRET//NOFORN       0.00      0.00      0.00        31
UNCLASSIFIED       0.79      0.85      0.82       802
UNCLASSIFIED//FOR OFFICIAL USE ONLY       1.00      0.07      0.14       284

avg / total       0.74      0.75      0.68      2002



/usr/lib/python2.7/dist-packages/sklearn/metrics/metrics.py:1905: UserWarning: The sum of true positives and false positives are equal to zero for some labels. Precision is ill defined for those labels [u'CONFIDENTIAL//NOFORN' u'SECRET' u'SECRET//NOFORN']. The precision and recall are equal to zero for some labels. fbeta_score is ill defined for those labels [u'CONFIDENTIAL//NOFORN' u'SECRET' u'SECRET//NOFORN']. 
  average=None)


In [65]:
vectorizer.fit_transform?

In [66]:
X_test

<501x26597 sparse matrix of type '<type 'numpy.float64'>'
	with 166104 stored elements in Compressed Sparse Row format>

In [68]:
X

<501x26150 sparse matrix of type '<type 'numpy.float64'>'
	with 160705 stored elements in Compressed Sparse Row format>